## **Ceyhun Onur**

## _18/10/2019_

# Task 1
Needed packages:

In [ ]:
installed.packages("data.table")

In [ ]:
library(data.table)

## Subtask 1

In [ ]:
# Read frome the snapshot, select only columns we're interested in.
matches <- fread(paste0(getwd(), "/data/matches.csv"), select=c("match_id", "match_hometeam_name", "match_awayteam_name", "league_id","match_status","match_hometeam_score", "match_awayteam_score"))
# get only finished english premier league matches
finished_epl_matches <- matches[league_id == 148 & match_status == "Finished"]
finished_epl_matches

# Task 2

In [ ]:
# read bets csv
bets <- fread(paste0(getwd(), "/data/bets.csv"))
# add column to indicate if match ended as draw
# get only bets data with win, draw or lose.
bets <- bets[variable %in% c("odd_1", "odd_x", "odd_2" )]
# join bets and match data by match_id
merged_data <- merge(finished_epl_matches, bets, by='match_id')
# find the most active bookmaker
bookmaker_data_count <- bets[, .N, by = c("odd_bookmakers")]
bookmaker_data_count <- bookmaker_data_count[order(-N)]
# select the first 4 bookmaker name that has most bets
bookmaker_names <- bookmaker_data_count[1:4]$odd_bookmakers
bookmaker_data_count
bookmaker_names

### Custom functions:

In [ ]:
# Use this function to analyze for different bookmakers.
bookmaker_draw_analysis <- function(bookmaker_name, data) {
  # get the data with selected bookmaker
  if(bookmaker_name == TRUE){
    bookmaker_bets <- data
  }
  else{
    bookmaker_bets <- data[odd_bookmakers == bookmaker_name]
  }
  # find probabilities from given odds
  bookmaker_bets <- bookmaker_bets[, c("p_win", "p_draw", "p_lose") := list(1/value[variable == 'odd_1'], 1/value[variable == 'odd_x'], 1/value[variable == 'odd_2']), by = list(match_id, odd_bookmakers, variable)]
  # group data by match_id and odd_bookmakers, so that we can have only one row for each match and bookmaker
  bookmaker_bets <- aggregate(bookmaker_bets[,c("p_win","p_draw","p_lose")], by=list(bookmaker_bets$match_id, bookmaker_bets$odd_bookmakers), FUN = function (x) first(na.omit(x)))
  # convert it back to data table
  bookmaker_bets <- data.table(bookmaker_bets)
  # rename grouped columns
  names(bookmaker_bets)[1] <- "match_id"
  names(bookmaker_bets)[2] <- "odd_bookmakers"
  # TASK 2.2
  # convert probabilities by implied probability function for each probability type
  bookmaker_bets[, p_win_imp:=p_win / (p_win + p_draw + p_lose)]
  bookmaker_bets[, p_draw_imp:=p_draw / (p_win + p_draw + p_lose)]
  bookmaker_bets[, p_lose_imp:=p_lose / (p_win + p_draw + p_lose)]
}

first_past_excluded <- bookmaker_draw_analysis(TRUE, merged_data)
bookmaker_names
first_past_excluded

In [ ]:
unplayed_epl_matches <- matches[league_id == 148 & is.na(match_status)]
unplayed_epl_matches <- unplayed_epl_matches[,-c("match_status", "match_hometeam_score", "match_awayteam_score", "league_id")]
unplayed_merged_data <- merge(unplayed_epl_matches, bets, by='match_id')
first_unplayed_predictions <- bookmaker_draw_analysis(TRUE, unplayed_merged_data)
first_unplayed_predictions <-  merge(unplayed_epl_matches, first_unplayed_predictions, by='match_id')
first_unplayed_predictions

In [ ]:
average_cols = c("p_win","p_draw","p_lose","p_win_imp","p_draw_imp","p_lose_imp")
average_predictions = first_unplayed_predictions[, lapply(.SD, mean), .SDcols = average_cols, by=match_id]
average_predictions = merge(unplayed_epl_matches, average_predictions, by='match_id')
average_predictions